<a href="https://colab.research.google.com/github/Tani-7/Omwansa-DS-portfolio/blob/Tani-7-ML/Financial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#pip install sec_edgar_downloader
%pip install yfinance

     |████████████████████████████████| 6.3 MB 4.9 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=0d15f4466e48774181617f12b3e074a91d0027c8a12f7e583911faca5749a2b1
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
from google.colab import drive
import os
import yfinance as yf
from sec_edgar_downloader import Downloader
import glob
import pandas as pd
import numpy as np
import datetime as datetime
import matplotlib.pyplot as plt 

%matplotlib inline

Files and Paths 


In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
with open('/content/drive/MyDrive/Financial Analysis/companies.txt') as f:
    companies = f.readlines()

In [5]:
date_after = "2000-01-01"

In [13]:
for company in companies:

    # Check if previously downloaded reports and skips if so
    check_path = glob.glob(f'sec-edgar-filings/{company}/*/*/*')
    if check_path != []:
        print(f"{company} 8-K & 10-K reports downloaded previously")
        continue

    # Initialize a downloader instance. 
    dl = Downloader()

    # Get all 8-K filings
    print(f"Downlading 8-K reports for {company}...") 
    dl.get("8-K", company, after=date_after)

    # Get all 10-K filings
    print(f"Downlading 10-K reports for {company}...") 
    dl.get("10-K", company, after=date_after)

Downlading 8-K reports for JPM
...
Skipping filing detail download for '0000019617-01-000004' due to network error: 404 Client Error: Not Found for url: https://www.sec.gov/Archives/edgar/data/0000019617/000001961701000004/0001.txt.
Skipping filing detail download for '0000019617-00-000167' due to network error: 404 Client Error: Not Found for url: https://www.sec.gov/Archives/edgar/data/0000019617/000001961700000167/0001.txt.
Skipping filing detail download for '0000019617-00-000159' due to network error: 404 Client Error: Not Found for url: https://www.sec.gov/Archives/edgar/data/0000019617/000001961700000159/0001.txt.
Skipping filing detail download for '0001007581-00-000019' due to network error: 404 Client Error: Not Found for url: https://www.sec.gov/Archives/edgar/data/0001007581/000100758100000019/0001.txt.
Skipping filing detail download for '0000019617-00-000132' due to network error: 404 Client Error: Not Found for url: https://www.sec.gov/Archives/edgar/data/0000019617/0000

KeyboardInterrupt: ignored

In [ ]:
Text Analysis on  downloaded statements

### Text Analysis on  downloaded statements

---



In [ ]:
strings = ["cybersecurity", "cyber security", "cyber-security", "information security", "data security", "data leak", "data breach"]
filings = ['8-K', '10-K']

df = pd.DataFrame(columns=['Company', 'Date', 'Filing', 'Word_count', *strings, 'Path'])

for company in companies:

    for filing in filings:

        paths = glob.glob(f'sec-edgar-filings/{company}/{filing}/*/*.txt')

        for path in paths:
            file = open(path, "r")

            # get date of file based on text at beginning of line
            for line in file:
                if "<SEC-DOCUMENT>" in line:
                    date = line[-9:-1]
                    break
            
            # counts if line contains any of the contents of strings
            # for line in file:
            #   if any(string in line.lower() for string in strings):
            #       string_count += 1

            total_count = 0
            word_count = {}
            for string in strings:
                word_count[string] = 0 
            for line in file:
                for string in strings:
                    word_count[string] += line.count(string)
                    total_count += line.count(string)

            word_counts = list(word_count.values())
            df.loc[len(df.index)] = [company, date, filing, total_count, *word_counts, path]
            file.close()

# convert date to datetime and sort in order
df['Date'] = pd.to_datetime(df['Date'],format='%Y%m%d')
df = df.sort_values(by='Date')

# Creates columns for 8-K & 10-K Counts
for filing in filings:  
    df[filing] = df.loc[(df['Filing'] == filing)]['Word_count']